#  Importing TMDB API

In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']
import os, time
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook


In [ ]:
#importing json librarie
import json
with open(r"\Users\camer\.secret\timdb_api.json", "r") as f:
    login = json.load(f)
    
login.keys()

In [ ]:
#test search to ensure success
tmdb.Movies(599).info()

# Creating and Defining Our Two Functions

In [ ]:
#define function for movie rating
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1']=='US':
            info['certification'] = c['certification']
    return info

In [ ]:
#first test of function
test1 = get_movie_with_rating('tt0848228')
test1

In [ ]:
#second test 
test2 = get_movie_with_rating('tt0332280')
test2

In [ ]:
#creating path to save data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

In [ ]:
#define function to make json files
def write_json(new_data, filename): 
  
    with open(filename,'r+') as file:
       
        file_data = json.load(file)
      
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
             file.seek(0)
       
             json.dump(file_data, file)

In [ ]:
#adding info for years 2000 and 2001
basics = pd.read_csv('Data/title_basics.csv.gz')
YEARS_TO_GET = [2000,2001]
errors = []
 

In [ ]:
#start with outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}], f)
    df = basics.loc[basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    #inner loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc = f'Movies from {YEAR}',
                                  position = 1,
                                  leave = True):
        try: 
            temp = get_movie_with_rating(movie_id)
            write_json(temp, JSON_FILE)
            time.sleep(0.02)
        except Exception as e:
            errors.append([movie_id, e])
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

print(f'total error {len(errors)}')
    
                                
                                          

# Exploratory Data Analysis

In [ ]:
#inspecting first df
df= pd.read_csv('DATA/final_tmdb_data_2000.csv.gz')
df.head()


In [ ]:
#inspecting second df
df2= pd.read_csv('DATA/final_tmdb_data_2001.csv.gz')
df2.head()

In [ ]:
#combining dataframes
final_df = pd.concat((df, df2), ignore_index=True)
display(final_df.head(),final_df.tail())

In [ ]:
#inspecting new datafram
final_df.info()

In [ ]:
#saving copy of new df
final_df.to_csv('Data/final_results_movie_00-01.csv.gz', compression='gzip',index=False)

In [ ]:
#only including movies with financial info more than 0
final_df = final_df.loc[final_df['budget']>0]
final_df = final_df.loc[final_df['revenue']>0] 
final_df.head()

In [1]:
#inspecting new data
final_df.info()

NameError: name 'final_df' is not defined

How many movies had at least some valid financial information (values > 0 for budget OR revenue)?

     - Well it seems after filtering all the values of '0' for Revenue and Budget we are left with 390 movies

How many movies are there in each of the certification categories (G/PG/PG-13/R)?

In [ ]:
#inspecting value counts for certification
final_df['certification'].value_counts()

In [ ]:
#creating bar graph to display count of each certification
#inspire from https://stackoverflow.com/questions/48939795/how-to-plot-a-count-bar-chart-with-a-pandas-df-grouping-by-one-categorical-colu
ax = final_df['certification'].value_counts().plot(kind='bar',
                                    figsize=(8,6),
                                    title="Number of Movies by Rating")
ax.set_xlabel("Movie Rating")
ax.set_ylabel("Frequency")
plt.show()

As seen above, most movies seems to be R and PG-13 while PG G and 'Not Rated' Movies are in the minority

What is the average revenue per certification category?

In [ ]:
#creating variable using groupby
certrev = final_df.groupby('certification')['revenue'].mean()
certrev

In [ ]:
#creating graph for said variable
plt.bar(certrev.index, certrev.values)
plt.title('Avergage Revenue by Rating')
plt.xlabel('Movie Rating')
plt.ylabel('Revenue in Millions')
plt.show()

While they make more R and PG-13 movies, G and PG make the most money possibly due to the higher audience pool

What is the average budget per certification category?

In [ ]:
#creating variable using groupby
certbudg = final_df.groupby('certification')['budget'].mean()
certbudg

In [ ]:
#creatinf final graph
plt.bar(certbudg.index, certbudg.values)
plt.title('Avergage  by Budget')
plt.xlabel('Movie Rating')
plt.ylabel('Budget in Millions')
plt.show()

It seems the movie rating that gets made the least, has the most budget. But, more money goes to PG-13 Movies than G.